First of all, we import all the necessary libraries

In [1]:
pip install torch numpy


Note: you may need to restart the kernel to use updated packages.


We load the GloVe embeddings we have downloaded

Load test set and validation set

In [4]:
pip show torchtext

Name: torchtext
Version: 0.15.1+cpu
Summary: Text utilities and datasets for PyTorch
Home-page: https://github.com/pytorch/text
Author: PyTorch core devs and James Bradbury
Author-email: jekbradbury@gmail.com
License: BSD
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy, requests, torch, torchdata, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install torchtext==0.10.0

ERROR: Could not find a version that satisfies the requirement torchtext==0.10.0 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.1, 0.15.2, 0.16.0, 0.16.1)
ERROR: No matching distribution found for torchtext==0.10.0


In [8]:
import pandas as pd
from torchtext.legacy.data import Field, LabelField, TabularDataset, BucketIterator
from torchtext.vocab import GloVe

file_path = "/kaggle/input/ys19-2023-assignment-2/train_set.csv"
valid_path = "/kaggle/input/ys19-2023-assignment-2/valid_set.csv"
glove_path = "/kaggle/input/glove-files/glove.twitter.27B.25d.txt"  

# Read the CSV files
train_data = pd.read_csv(file_path)
valid_data = pd.read_csv(valid_path)

TEXT = Field(tokenize='spacy', include_lengths=True)
LABEL = LabelField(dtype=torch.float)

fields = [('text', TEXT), ('label', LABEL)]
train_examples = [(train_data['text'][i], train_data['label'][i]) for i in range(len(train_data))]
valid_examples = [(valid_data['text'][i], valid_data['label'][i]) for i in range(len(valid_data))]
train_data, valid_data = TabularDataset.splits(path='', train=train_examples, validation=valid_examples, format='csv', fields=fields)
TEXT.build_vocab(train_data, max_size=50000, vectors=GloVe(name='twitter.27B.25d', cache=glove_path))
LABEL.build_vocab(train_data)

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data),
    batch_size=64,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    device=device
)

ModuleNotFoundError: No module named 'torchtext.legacy'

In [11]:
import re

def tokenize(input_text):
    eyes = "[8:=;]"
    nose = "['`\-]?"

    input_text = re.sub(r'https?://\S+\b|www\.(\w+\.)+\S*', "<URL>", input_text)
    input_text = re.sub(r'/', ' / ', input_text)  # Force splitting words appended with slashes
    input_text = re.sub(r'@\w+', "<USER>", input_text)
    input_text = re.sub(rf'{eyes}{nose}[)d]+|[)d]+{nose}{eyes}', "<SMILE>", input_text, flags=re.IGNORECASE)
    input_text = re.sub(rf'{eyes}{nose}p+', "<LOLFACE>", input_text, flags=re.IGNORECASE)
    input_text = re.sub(rf'{eyes}{nose}\(+|\)+{nose}{eyes}', "<SADFACE>", input_text)
    input_text = re.sub(rf'{eyes}{nose}[\/|l*]', "<NEUTRALFACE>", input_text)
    input_text = re.sub(r'<3', "<HEART>", input_text)
    input_text = re.sub(r'[-+]?[.\d]*[\d]+[:,.\d]*', "<NUMBER>", input_text)
    input_text = re.sub(r'#\S+', lambda hashtag: "<HASHTAG> " + " ".join(hashtag[1:].split(r'(?=[A-Z])')), input_text)
    input_text = re.sub(r'([!?.]){2,}', lambda punct: punct.group(1) + " <REPEAT>", input_text)
    input_text = re.sub(r'\b(\S*?)(.)\2{2,}\b', lambda word: word.group(1) + word.group(2) + " <ELONG>", input_text)
    input_text = re.sub(rf'([^a-z0-9()<>\'`\-]){2,}', lambda caps: caps.group(0).lower() + " <ALLCAPS>", input_text)

    return input_text


In [15]:
import torch
import torchtext
from torchtext.vocab import GloVe
import pandas as pd
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader

embed_len = 300
def vectorize_batch(batch):
    Y, X = list(zip(*batch))
    X = [tokenize(x) for x in X]
    X = [tokens+[""] * (max_words-len(tokens))  if len(tokens)<max_words else tokens[:max_words] for tokens in X]
    X_tensor = torch.zeros(len(batch), max_words, embed_len)
    for i, tokens in enumerate(X):
        X_tensor[i] = global_vectors.get_vecs_by_tokens(tokens)
    return X_tensor.reshape(len(batch), -1), torch.tensor(Y) - 1
target_classes = ["Positive", "Neutral", "Negative"]
file_path = "/kaggle/input/ys19-2023-assignment-2/train_set.csv"
valid_path = "/kaggle/input/ys19-2023-assignment-2/valid_set.csv"
train_dataset = pd.read_csv(file_path)
test_dataset = pd.read_csv(valid_path)
train_dataset, test_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)
train_loader = DataLoader(train_dataset, batch_size=1024, collate_fn=vectorize_batch)
test_loader  = DataLoader(test_dataset, batch_size=1024, collate_fn=vectorize_batch)
for X, Y in train_loader:
    print(X.shape, Y.shape)
    break

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_47/2288897665.py", line 25, in <module>
    for X, Y in train_loader:
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 634, in __next__
    self._num_yielded > self._IterableDataset_len_called:
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 678, in _next_data
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/tmp/ipykernel_47/2288897665.py", line 10, in vectorize_batch
    Y, X = list(zip(*batch))
ValueError: too many values to unpack (expected 2)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/c